# E7 - Project
Team member -
Rutuparn
Tanish
Vihang
Samarpit

# Reading images from the folder


In [2]:
import cv2
import os
import numpy as np

def load_and_process_images(folder_path='images', target_size=(800, 600)):
    """
    Loads all images from a folder.
    - Crops to 4:3 aspect ratio if needed.
    - Scales down to 800x600 if larger, but never scales up.
    - Draws 8x8 grid lines.
    - Saves to 'resized_images'.
    - Divides into 8x8 tiles (for further processing).
    """
    supported_extensions = ('.jpg', '.jpeg', '.png', '.bmp', '.tiff')
    images = []
    resized_images = []
    divided_images = []

    # Create output folder
    output_folder = 'resized_images'
    os.makedirs(output_folder, exist_ok=True)

    if not os.path.exists(folder_path):
        raise FileNotFoundError(f"The folder '{folder_path}' does not exist.")

    for filename in os.listdir(folder_path):
        if filename.lower().endswith(supported_extensions):
            file_path = os.path.join(folder_path, filename)
            try:
                img = cv2.imread(file_path)
                if img is None:
                    print(f"⚠️ Skipped unreadable image: {filename}")
                    continue

                h, w = img.shape[:2]
                aspect_ratio = w / h

                # --- a. Enforce 4:3 aspect ratio by cropping ---
                target_ratio = 4 / 3
                if abs(aspect_ratio - target_ratio) > 0.01:  # allow slight tolerance
                    if aspect_ratio > target_ratio:
                        # Image is too wide → crop width
                        new_w = int(h * target_ratio)
                        x_start = (w - new_w) // 2
                        img = img[:, x_start:x_start + new_w]
                    else:
                        # Image is too tall → crop height
                        new_h = int(w / target_ratio)
                        y_start = (h - new_h) // 2
                        img = img[y_start:y_start + new_h, :]

                # --- b. Scale down if larger than 800x600 ---
                h, w = img.shape[:2]
                if w > target_size[0] or h > target_size[1]:
                    img = cv2.resize(img, target_size, interpolation=cv2.INTER_AREA)

                # --- c. Do NOT scale up smaller images ---
                final_img = img
                h, w = final_img.shape[:2]

                # Draw 8x8 grid lines
                tile_h, tile_w = h // 8, w // 8
                for i in range(1, 8):
                    # Horizontal lines
                    y = i * tile_h
                    cv2.line(final_img, (0, y), (w, y), (0, 255, 0), 1)
                    # Vertical lines
                    x = i * tile_w
                    cv2.line(final_img, (x, 0), (x, h), (0, 255, 0), 1)

                # Save processed image
                save_path = os.path.join(output_folder, filename)
                cv2.imwrite(save_path, final_img)

                # Keep copies in memory
                images.append(final_img.copy())
                resized_images.append(final_img)

                # Divide into 8x8 tiles (using actual dimensions)
                tiles = [
                    final_img[i * tile_h:(i + 1) * tile_h, j * tile_w:(j + 1) * tile_w]
                    for i in range(8)
                    for j in range(8)
                ]
                divided_images.append(tiles)

            except Exception as e:
                print(f"❌ Error processing {filename}: {e}")

    return images, resized_images, divided_images


# ----------------------------
# Example usage
# ----------------------------
if __name__ == "__main__":
    images, resized_images, divided_images = load_and_process_images('images')

    print(f"✅ Loaded and processed {len(images)} images.")
    print(f"📁 Grid-marked images saved in the 'resized_images' folder.")
    if divided_images:
        print(f"Each image divided into {len(divided_images[0])} tiles (8x8).")


✅ Loaded and processed 465 images.
📁 Grid-marked images saved in the 'resized_images' folder.
Each image divided into 64 tiles (8x8).


# Conversion into 800x600 pixels


# dividing into 8x8 grid

Converting the images to greycode to reduce dimentionality

# Image processing to identify animals using HOG.

In [ ]:
import cv2
import os
import numpy as np

def safe_imread(file_path):
    """
    Safely read image using OpenCV with memory protection.
    Falls back to reduced-size reads if memory allocation fails.
    """
    try:
        img = cv2.imread(file_path)
        if img is None:
            return cv2.imread(file_path, cv2.IMREAD_REDUCED_COLOR_2)
        return img
    except Exception:
        return cv2.imread(file_path, cv2.IMREAD_REDUCED_COLOR_4)


def crop_to_aspect_ratio(img, target_aspect=(4, 3)):
    """
    Crop the image to a target aspect ratio (4:3 by default).
    """
    h, w = img.shape[:2]
    target_ratio = target_aspect[0] / target_aspect[1]
    current_ratio = w / h

    if abs(current_ratio - target_ratio) < 1e-2:
        return img  # Already close enough

    if current_ratio > target_ratio:
        # Too wide — crop horizontally
        new_w = int(h * target_ratio)
        start_x = (w - new_w) // 2
        cropped = img[:, start_x:start_x + new_w]
    else:
        # Too tall — crop vertically
        new_h = int(w / target_ratio)
        start_y = (h - new_h) // 2
        cropped = img[start_y:start_y + new_h, :]

    return cropped


def resize_image(img, max_size=(800, 600)):
    """
    Resize the image to a maximum of 800x600.
    Do not scale up smaller images.
    """
    h, w = img.shape[:2]
    max_w, max_h = max_size
    if w > max_w or h > max_h:
        return cv2.resize(img, (max_w, max_h), interpolation=cv2.INTER_AREA)
    return img


def draw_grid(img, grid_size=(8, 8)):
    """
    Draw an 8x8 grid and small numbered labels on the image.
    """
    h, w = img.shape[:2]
    grid_h, grid_w = grid_size
    dy, dx = h // grid_h, w // grid_w
    numbered = img.copy()

    cell_num = 1
    for y in range(0, h, dy):
        for x in range(0, w, dx):
            # Draw grid lines
            cv2.rectangle(numbered, (x, y), (x + dx, y + dy), (0, 255, 0), 1)
            # Draw smaller number at bottom-left corner of each cell
            cv2.putText(numbered, str(cell_num), (x + 5, y + dy - 5),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.3, (0, 0, 255), 1, cv2.LINE_AA)
            cell_num += 1

    return numbered


def load_and_process_images(folder_path='images', output_folder='resized_images'):
    """
    Load all valid images, process, grid, and save to new folder.
    """
    supported_extensions = ('.jpg', '.jpeg', '.png', '.bmp', '.tiff')
    os.makedirs(output_folder, exist_ok=True)

    for filename in os.listdir(folder_path):
        if not filename.lower().endswith(supported_extensions):
            continue

        file_path = os.path.join(folder_path, filename)
        img = safe_imread(file_path)
        if img is None:
            print(f"⚠️ Skipped unreadable image: {filename}")
            continue

        # Crop to 4:3 aspect ratio
        img = crop_to_aspect_ratio(img, target_aspect=(4, 3))

        # Resize down if needed (don’t upscale)
        img = resize_image(img, max_size=(800, 600))

        # Draw grid + numbering
        grid_img = draw_grid(img, grid_size=(8, 8))

        # Save to output folder
        save_path = os.path.join(output_folder, filename)
        cv2.imwrite(save_path, grid_img)
        print(f"✅ Processed and saved: {filename}")


# Example usage
if __name__ == "__main__":
    load_and_process_images('images', 'resized_images')


ℹ️ Not scaling up smaller image: CIMG0117.JPG
ℹ️ Not scaling up smaller image: CIMG0120(1).JPG
ℹ️ Not scaling up smaller image: CIMG0144~3.JPG
ℹ️ Not scaling up smaller image: CIMG0209.JPG
ℹ️ Not scaling up smaller image: CIMG0210.JPG
ℹ️ Not scaling up smaller image: CIMG0218.JPG
ℹ️ Not scaling up smaller image: CIMG0228(1).JPG
ℹ️ Not scaling up smaller image: CIMG0230~2.JPG
ℹ️ Not scaling up smaller image: CIMG0232~2.JPG
ℹ️ Not scaling up smaller image: CIMG0233.JPG


error: OpenCV(4.12.0) D:\a\opencv-python\opencv-python\opencv\modules\core\src\alloc.cpp:73: error: (-4:Insufficient memory) Failed to allocate 42467328 bytes in function 'cv::OutOfMemoryError'
